In [2]:
import numpy as np 
import random
import copy

# Creating a MDP

In [3]:
## 1question creating a MDP 
def createMDP ( probSuccess , gridSize ) :
    #Number of actions
    k  = 4 ;  
    #set of actions L,R,U,D
    actionLiist = [ 0, 1, 2, 3]  ; 
    # defining probability tranisition matrix
    probTransitionMatrix = np.zeros((gridSize , gridSize ,gridSize , gridSize , k )) ;
    n = gridSize ;
    p = probSuccess ;
    # assuming successful action takes to right state with probability p
    # and unsuccessful action remains in the same state with probabiliy (1-p)
    for i in range(0,n) :
        for j in range(0,n) :
            for k in range(0,4) :
                probTransitionMatrix[i][j][i][j][k] = 1 - p ;
                if( k == 0 ) :
                    if(j > 0 ) :
                        probTransitionMatrix[i][j][i][j-1][k] = p ;
                    else :
                        probTransitionMatrix[i][j][i][j][k] = 1 ;
                if ( k == 1 ) :
                    if(j < (n-1) ) :
                        probTransitionMatrix[i][j][i][j+1][k] = p ;
                    else :
                        probTransitionMatrix[i][j][i][j][k] = 1 ;
                if ( k == 2 ) :
                    if(i > 0 ) :
                        probTransitionMatrix[i][j][i-1][j][k] = p ;
                    else :
                        probTransitionMatrix[i][j][i][j][k] = 1 ;
                if ( k == 3 ) :
                    if(i < (n-1) ) :
                        probTransitionMatrix[i][j][i+1][j][k] = p ;
                    else :
                        probTransitionMatrix[i][j][i][j][k] = 1 ;
    #initlaising reward matrix ...
    rewardMatrix = np.zeros((n,n)) ;
    rewardMatrix[n-1][n-1] = 100 ;
    rewardMatrix[0][0] = 10 
    return  (probTransitionMatrix , rewardMatrix) ; 

# Random SDP

In [4]:
## questionNo.2
## random SDP 
def randomSDP (MDP) :
    #size of MDP 
    n = MDP[0].shape[0] ; 
    policy = np.zeros((n,n)) ;
    action = [0, 1, 2, 3] ;
    for i in range(0,n) :
        for j in range (0,n) :
            policy[i][j] = int(random.choice(action)) ;
    return policy ; 

# Gauss Elimination 

In [5]:
## questionNo.3 
## Gauss Elimination 
def gaussElimination(a, b):
    a = copy.deepcopy(a)
    b = copy.deepcopy(b)
    n = len(a)
    p = len(b[0])
    det = 1
    for i in range(n - 1):
        k = i
        for j in range(i + 1, n):
            if abs(a[j][i]) > abs(a[k][i]):
                k = j
        if k != i:
            a[i], a[k] = a[k], a[i]
            b[i], b[k] = b[k], b[i]
            det = -det
 
        for j in range(i + 1, n):
            t = a[j][i]/a[i][i]
            for k in range(i + 1, n):
                a[j][k] -= t*a[i][k]
            for k in range(p):
                b[j][k] -= t*b[i][k]
 
    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n):
            t = a[i][j]
            for k in range(p):
                b[i][k] -= t*b[j][k]
        t = 1/a[i][i]
        det *= a[i][i]
        for j in range(p):
            b[i][j] *= t
    return (det, b)

In [14]:
def Hash( i , j , n) :
    return int(i*n + j) ;

# Value via inverse

In [7]:
## calculatining Vu 
## question No. 4 
def valueviainverse ( prob , rew , gamma) :  
    MDP = (prob , rew) ;
    policy = randomSDP(MDP) ;
    n = MDP[0].shape[0] ;
    probTrans = MDP[0] ;
    P =  np.zeros((n*n,n*n)) ;
    R =  np.zeros((n*n))
    for i in range(0,n) :
        for j in range(0,n) :
            R[Hash(i,j,n)] = rew[i][j] ;
            k = int(policy[i][j]) ;
            P[Hash(i,j,n)][Hash(i,j,n)] = probTrans[i][j][i][j][k]; 
            if( k == 0 ) :
                if(j > 0 ) :
                    P[Hash(i,j,n)][Hash(i,j-1,n)] = probTrans[i][j][i][j-1][k];
            if ( k == 1 ) :
                if(j < (n-1) ) :
                    P[Hash(i,j,n)][Hash(i,j+1,n)] = probTrans[i][j][i][j+1][k] ; 
            if ( k == 2 ) :
                if(i > 0 ) :
                    P[Hash(i,j,n)][Hash(i-1,j,n)] = probTrans[i][j][i-1][j][k];
            if ( k == 3 ) :
                if(i < (n-1) ) :
                    P[Hash(i,j,n)][Hash(i+1,j,n)] = probTrans[i][j][i+1][j][k];
    P = gamma*P ;
    I = np.identity(n*n) ;
    Inv = np.subtract(I , P) ;
    Inv  = np.linalg.inv(Inv) ;
    V = Inv.dot(R) ;
    Vu = np.zeros((n,n)) ;
    for i in range(0,n*n) :
        y = i % n ;
        x = (i - y) // n ;
        Vu[x][y] = V[i] ;         
    return Vu ;

# Bellman operator

In [ ]:
## question No 5 
## bellman operator 
def BellmanOperator(V , P , R , Pu , gamma) :
    Pu = gamma*Pu ;
    V = R + Pu.dot(V) ;
    return V ;

# Value Evaluation

In [9]:
## question No 6 
def valueEvaluation(prob , rew , gamma) :
    MDP = (prob , rew) ;
    policy = randomSDP(MDP) ;
    n = MDP[0].shape[0] ;
    probTrans = MDP[0] ;
    P =  np.zeros((n*n,n*n)) ;
    R =  np.zeros((n*n))
    for i in range(0,n) :
        for j in range(0,n) :
            R[Hash(i,j,n)] = rew[i][j] ;
            k = int(policy[i][j]) ;
            P[Hash(i,j,n)][Hash(i,j,n)] = probTrans[i][j][i][j][k]; 
            if( k == 0 ) :
                if(j > 0 ) :
                    P[Hash(i,j,n)][Hash(i,j-1,n)] = probTrans[i][j][i][j-1][k];
            if ( k == 1 ) :
                if(j < (n-1) ) :
                    P[Hash(i,j,n)][Hash(i,j+1,n)] = probTrans[i][j][i][j+1][k] ; 
            if ( k == 2 ) :
                if(i > 0 ) :
                    P[Hash(i,j,n)][Hash(i-1,j,n)] = probTrans[i][j][i-1][j][k];
            if ( k == 3 ) :
                if(i < (n-1) ) :
                    P[Hash(i,j,n)][Hash(i+1,j,n)] = probTrans[i][j][i+1][j][k];
    
    V = np.zeros((n*n)) ;       
    Vu = np.zeros((n,n)) 
    
    for i in range(1000) :
        V = BellmanOperator(V,P,R,P,gamma) ;    
    for i in range(0,n*n) :
        y = i % n ;
        x = (i - y) // n ;
        Vu[x][y] = V[i] ;         
        
    return Vu ;

# Random Walk

In [10]:
#question No.7 
#RandomWalk
def RandomWalk(L, policy , P , R) : 
    
    j,k = 1,1 ;
    
    n = R.shape[0] ;
    
    for i in range(L) :
        k1 = int(policy[j][k]) ;
        l = [] ;
        l1 = [] ;
        for i1 in range(n) :
            for j1 in range(n) :
                l.append(P[k][j][i1][j1][k1]) ;
                l1.append(i1*n+j1) ;
        x1 = np.random.choice(l1,p=l) ;
        j1 = x1 % n ;
        k1 = (x1 - j1) // n ;
        print (" ( ({}, {}) , {} , ({}, {}) )".format(j,k,R[j][k],j1,k1)) ;
        j = j1 ;
        k = k1 ;
        

# Testing

In [13]:
#testing 
MDP = createMDP(0.5, 3) ;
prob = MDP[0] ;
rew = MDP[1] ;
pol = randomSDP(MDP) ;
print("Printing the value matrix using a random policy generated by SDP and also via inverse method")
print(valueviainverse(prob,rew,0.9)) ;
print("Printing the value matrix using a random policy generated by SDP and also via iteration method")
print(valueEvaluation(prob,rew,0.9)) ;
print("Random Walk")
RandomWalk(20,pol,prob,rew)

Printing the value matrix using a random policy generated by SDP and also via inverse method
[[ 100.    0.    0.]
 [   0.    0.    0.]
 [   0.    0. 1000.]]
Printing the value matrix using a random policy generated by SDP and also via iteration method
[[ 100.    0.    0.]
 [   0.    0.    0.]
 [   0.    0. 1000.]]
Random Walk
 ( (1, 1) , 0.0 , (1, 0) )
 ( (1, 0) , 0.0 , (1, 0) )
 ( (1, 0) , 0.0 , (1, 0) )
 ( (1, 0) , 0.0 , (1, 0) )
 ( (1, 0) , 0.0 , (1, 0) )
 ( (1, 0) , 0.0 , (1, 0) )
 ( (1, 0) , 0.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
 ( (0, 0) , 10.0 , (0, 0) )
